# Rust 基本文法

## 変数宣言

### let, mut
値を変数に束縛するときは `let` 宣言を用いる

Rustでは、変数はデフォルトで変更不可であるため、変数を変更可能にしたい場合は `mut` を付けて変数宣言する必要がある

In [2]:
// 変数を束縛する際、多くの場合は型を明示しなくてもコンパイラが自動的に型を推論してくれる
let immutable_value = 10; // i32型 に自動推論される

// 変更可能な変数を宣言
let mut mutable_value = 20;

// 変更可能な変数には、同じ型の別値を再代入可能
mutable_value += immutable_value; // mutable_value = mutable_value(=20) + immutable_value(=10) => 30

// 型を明示する場合
let u32val: u32 = mutable_value; // u64型 の変数に束縛

// 数値限定で、値の後ろに直接型名を付けて指定することもできる
let u64val = 1234u64; // u64型 の 1234

### const, static
`const` と `static` はいずれも定数を宣言するためのキーワード

これらは、グローバルスコープでも特定のブロック内スコープでも、任意の場所で利用することができる

`const` は常に変更不可であるため、別の値を紐付けたり変更したりすることはできない

一方 `static` は変更可能であるため、グローバルスコープで定義した static の値は、どこからでも変更可能な危険な変数になってしまう

そのため、グローバル static 値は `unsafe` ブロックに入れない限り操作することはできなくなっている

In [3]:
// PI定数の定義
// => コンパイル時に PI は 3.14 という実際の値に置き換えられる
const PI: f32 = 3.14; // PI を別の値に変更することは出来ない

// 'static ライフタイムを持つ定数を作成
// => バイナリの特定のセクションに配置され、プログラム中最長のライフタイムを持つ
static NUMBER: i32 = 18;

## 制御構文

### if式
条件分岐を行う際は `if` キーワードに条件を渡してプログラムを分岐する

Rustにおける if は式であるため、評価した値を変数に束縛したり、関数の引数にすることもできる

ただし、if式が返す値の型は全て揃っている必要がある

In [4]:
fn main() {
    let number: i32 = 1;
    let answer: &str = if number < 0 {
        "負の値です" // if式 で値を返す際は ; をつけてはならない（; をつけると文になってしまう）
    } else if number == 0 {
        "ゼロです"
    } else {
        "正の値です"
    };
    
    println!("{}", answer);
}
main();

正の値です


### loop, while, for
Rustにはループ処理を行うための制御構文として `loop`, `while`, `for` の3つが存在する

この内 `loop` は式であり `break` キーワードに戻り値を付与して値を返すことができる

式と文という違いはあるが、基本的な挙動は `loop` と `while true` で同じであり、どちらも無限ループとなる（ただし `while` は式ではないため値を返すことはない）

In [5]:
fn main() {
    let mut count = 0;
    let result = loop {
        count += 1; // Rust にはインクリメント構文 (++) はない
        if count == 10 { // count が 10 になったタイミングで抜ける
            break count; // count 値を戻り値として返す
        }
    };
    
    println!("{}", result); // => 10
}
main();

10


In [6]:
fn main() {
    let mut count = 0;
    
    // count が 10 未満である限りループする
    while count < 10 {
        count += 1;
    }
    
    println!("{}", count);
}
main();

10


In [7]:
fn main() {
    // 0..10 の範囲で繰り返す (ただし 10 は含まれない)
    // Range型: (start)..(end + 1)
    for count in 0..10 {
        println!("count: {}", count);
    }
    
    // 配列を反復子（イテレータ）として繰り返し処理
    let fruits = ["apple", "banana", "orange"];
    for fruit in &fruits {
        println!("fruit: {}", fruit);
    }
}
main();

count: 0
count: 1
count: 2
count: 3
count: 4
count: 5
count: 6
count: 7


In [8]:
// loop, while, for にはラベルを付けることができ、break 時にそのラベルを指定して繰り返しを抜けることができる
// ラベル名はその先頭に ' を付ける必要がある

// mainラベル: 0 ~ 9 for-loop
'main: for i in 0..10 {
    println!("main loop start: {}", i);
    
    // subラベル: loop
    'sub: loop {
        println!("sub loop start");
        
        // mainラベルのループを抜ける
        break 'main;
        
        println!("sub loop end"); // => 表示されない
    }
    println!("main loop end"); // => 表示されない
}

count: 8
count: 9
fruit: apple
fruit: banana
fruit: orange
main loop start: 0
sub loop start


()

#### Range
forループで特定の範囲の数値を指定するときは Range型 を使う

Range型の数値範囲指定は以下のようになっている

- `S..E`: 数値S以上、数値E未満の範囲
- `S..=E`: 数値S以上、数値E以下の範囲

In [9]:
// 1 ~ 2 loop
for i in 1..3 {
    println!("loop1: {}", i);
}

// 1 ~ 3 loop
for i in 1..=3 {
    println!("loop2: {}", i);
}

loop1: 1
loop1: 2
loop2: 1
loop2: 2
loop2: 3


()

#### Iterator
forループでは、データの集合から1つずつ要素を取り出して順番に繰り返し処理ができる

これは、データの集合に Iteratorトレイト が実装されているためである

ユーザ定義型にも Iteratorトレイト を適用でき、それによりforループで繰り返しすることができるようになる

※ トレイトについては後述

Iteratorトレイトを適用するには以下の2つの操作を行う必要がある

- Iteratorが出力する型を決定し type Item に紐付ける
- Iteratorの次の要素を返すための next() メソッドを実装する

In [10]:
// 自作型: MyIter構造体
struct MyIter {
    current: usize,
    max: usize,
}

// MyIter構造体に Iteratorトレイト 適用
impl Iterator for MyIter {
    type Item = usize; // Iteratorが出力する型を usize に決定
    
    // next() メソッドの定義
    fn next(&mut self) -> Option<usize> {
        let next_value = 
            // 現在位置が max 以下なら現在位置を返す
            if self.current <= self.max {
                Some(self.current)
            } else {
                None
            };
        self.current += 1; // インクリメント
        return next_value;
    }
}

fn main() {
    let it = MyIter {
        current: 0,
        max: 10,
    };
    // println: 0 ~ 10
    for num in it {
        println!("{}", num);
    }
}
main();

0
1
2
3
4
5


### match
C言語などのプログラミング言語には、変数の値によって処理を分岐させる `switch` 構文がある

Rustにも同じように分岐を行うことができる `match` 式が存在し、これはC言語の `switch` 構文よりも強力なパターンマッチングを行うことができる

ここで、Rustにおける「パターン」とは、型の構造に一致しているか確認するための記法を指す

パターンは、数値や文字列のような単純な値だけでなく、列挙型やタプル、構造体などの比較を行ったり、値の範囲比較やワイルドカードによるマッチングも行うことができる

In [11]:
fn main() {
    // 単純な数値マッチング
    let i: i32 = 1;
    match i {
        1 => println!("One"),
        2 => println!("Two"),
        3 => println!("Three"),
        _ => println!("Misc"), // アンダースコアはあらゆる値にマッチするワイルドカード
    } // => One
}
main();

6
7
8
9
10
One


In [12]:
fn main() {
    // 列挙型のマッチング
    enum Color {
        Red,
        Blue,
        Green,
    }
    
    let c = Color::Red;
    
    // 列挙型を match で分岐する際は、網羅性の確認をし、全ての列挙子に対する処理が存在するかチェックする
    // => 足りない列挙子があった場合はコンパイルエラーとなる
    match c {
        Color::Red => println!("Red"),
        Color::Blue => println!("Blue"),
        // Color::Green に対する処理が抜けている => コンパイルエラー
    }
}
main();

Error: non-exhaustive patterns: `Green` not covered

In [13]:
fn main() {
    let result: Result<i32, String> = Err("I'm error result".to_string());
    
    // match は式であるため、分岐処理の結果を変数に束縛することができる
    let code: i32 = match result {
        Ok(number) => number,
        Err(message) => { // match 分岐処理を複数記述したい場合は {} で囲む
            println!("Error: {}", message);
            -1 // Rust は最後に評価した式は自動的に return される
        },
    };
    
    println!("Result: {}", code);
}
main();

Error: I'm error result
Result: -1
